In [110]:
import petl as etl
import pandas as pd

In [111]:
# Read files

In [112]:
fire_station = (
                etl
                .fromcsv('data/Fire_Stations.csv')
                )
station_locations = etl.fromxml('data/Station_Locations.xml','STATION', 
                                {
                                    'NAME': 'NAME',
                                    'LONG':'LONG',
                                    'LAT':'LAT'
                                })
station_regions = etl.fromcsv('data/Station_Regions.csv')


In [113]:
#1.Clean non-printable unicode characters

In [114]:
fire_station["Fire Station"]

fire_station_df = fire_station.todataframe()
fire_station_df['Fire Station'] = fire_station_df['Fire Station'].str.encode('ascii', 'ignore').str.decode('ascii')
#fire_station_df["Phone Number"]

In [115]:
#2.Construct email

In [116]:
fire_station_df['email'] = 'enquire@' + fire_station_df['Fire Station'].str.replace('Fire Station','').str.replace(' ','').str.lower() + '.qfes.gov.au'

In [117]:
#3a.Prepare datasets for joins

In [118]:

#station_locations = etl.rename(station_locations, {'NAME': 'Region'})
station_locations_df = station_locations.todataframe()
station_locations_df.head()

,LAT,LONG,NAME
0,-27.58115886,153.0385326,ACACIA RIDGE
1,-20.27804421,148.7274619,AIRLIE BEACH
2,-28.03477015,151.984646,ALLORA
3,-23.64926834,146.6332709,ALPHA
4,-27.3984497,153.4422231,AMITY POINT


In [119]:
#station_regions = etl.rename(station_regions, {'Region Name': 'Region'})
station_regions_df = station_regions.todataframe()
#station_regions_df

In [120]:
fire_station_df['Region'] = fire_station_df['Region'].astype('str') 
fire_station_df['Name'] = fire_station_df['Fire Station'].str.replace(' Fire Station','').str.upper()
#fire_station_df

In [121]:
#3b.Join 3 datasets to prepare desired table

In [122]:
#etl.leftjoin(fire_station_df, station_locations, key='NAME')
#JOIN doesn't work with "petl.transform.headers.RenameView & pandas.core.frame.DataFrame"


In [123]:
#Join fire station and station location data sets
join_df_1 = pd.merge(fire_station_df, station_locations_df, left_on='Name',right_on='NAME'
                     ,how='left',suffixes=('_left','_right'))

#Join fire station & station location with station regions data set.
join_df_2 = pd.merge(join_df_1, station_regions_df, left_on='Region',right_on='Region Code'
                     ,how='left',suffixes=('_left','_right'))


In [124]:
#4, 5 : Follow Field names like Table 1 - Field Mapping
columns = ['RegionID', 'Stn Number', 'Fire Station', 'Stn Type', 'Address', 'Phone Number', 'Fax Number', 'email','LAT','LONG' ]
final_df = join_df_2[columns]


In [125]:
#6: Sort data by “RegionID” and then on “Station Number”

In [127]:
final_df['RegionID'] = final_df['RegionID'].astype('int')
final_df['Stn Number'] = final_df['Stn Number'].astype('int') 
final_df.sort_values(['RegionID', 'Stn Number'], ascending='True')
final_df.dtypes

/Users/raj/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/raj/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


RegionID         int64
Stn Number       int64
Fire Station    object
Stn Type        object
Address         object
Phone Number    object
Fax Number      object
email           object
LAT             object
LONG            object
dtype: object

In [128]:
print(final_df)

     RegionID  Stn Number                Fire Station              Stn Type  \
0           1           1     Kemp Place Fire Station            ​Permanent   
1           1           2    Roma Street Fire Station            ​Permanent   
2           1           3       Annerley Fire Station            ​Permanent   
3           1           4        Windsor Fire Station            ​Permanent   
4           1           5       Ashgrove Fire Station            ​Permanent   
5           1           6    Redland Bay Fire Station            ​Permanent   
6           1           7    Cannon Hill Fire Station            ​Permanent   
7           1           8       Enoggera Fire Station            ​Permanent   
8           1           9      Camp Hill Fire Station            ​Permanent   
9           1          10   Acacia Ridge Fire Station            ​Permanent   
10          1          11         Hendra Fire Station            ​Permanent   
11          1          12        Taringa Fire Statio

In [30]:
from postal.expand import expand_address
from postal.parser import parse_address


In [35]:
def parse_address_append()
    abcd = parse_address('236 Lloyd St Enoggera 4051')

In [59]:
abcd


[('236', 'house_number'),
 ('lloyd st', 'road'),
 ('enoggera', 'suburb'),
 ('4051', 'postcode')]

In [62]:
final_df['Street Address'] = final_df['Address'].apply(lambda x: parse_address(x)[1][0])

/Users/raj/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [101]:


def split_address_after_parsing(address,index):
    if address is not '':
        address_ls = parse_address(address)
        if index < len(address_ls):
            return address_ls[index][0]
        else:
            return ''
    return ''
    

#Follow below index values for parsing method called 'split_address_after_parsing'
#0:house_number, 1:road, 2:suburb, 3:postcode   
final_df['Suburb'] = final_df['Address'].apply(lambda x: split_address_after_parsing(x,2))
final_df['Post Code'] = final_df['Address'].apply(lambda x: split_address_after_parsing(x,3))

/Users/raj/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/Users/raj/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [103]:
final_df

,RegionID,Stn Number,Fire Station,Stn Type,Address,Phone Number,Fax Number,email,LAT,LONG,suburb,Street Address,Suburb,Post Code
0,1,1,Kemp Place Fire Station,​Permanent,21 Martin Street Fortitude Valley 4006,​07 3362 9923,,enquire@kempplace.qfes.gov.au,-27.46061736,153.0357795,martin street,martin street,fortitude valley,4006
1,1,2,Roma Street Fire Station,​Permanent,279 Upper Roma St Brisbane City 4101,​07 3247 5538,,enquire@romastreet.qfes.gov.au,-27.46581599,153.0153914,upper roma st,upper roma st,brisbane city,4101
2,1,3,Annerley Fire Station,​Permanent,346 Ipswich Rd Annerley 4103,​07 3247 5210,,enquire@annerley.qfes.gov.au,-27.50439112,153.0347521,ipswich rd,ipswich rd,annerley,4103
3,1,4,Windsor Fire Station,​Permanent,7 Truro St Windsor 4030,​07 3247 5564,,enquire@windsor.qfes.gov.au,-27.42771528,153.0331964,truro st,truro st,windsor,4030
4,1,5,Ashgrove Fire Station,​Permanent,515 Waterworks Rd Ashgrove 4060,​07 3366 0258,,enquire@ashgrove.qfes.gov.au,-27.44789239,152.9760243,waterworks rd,waterworks rd,ashgrove,4060
5,1,6,Redland Bay Fire Station,​Permanent,33 Gordon Road Redland Bay 4165,​07 3829 1403,,enquire@redlandbay.qfes.gov.au,-27.61089735,153.2890479,gordon road,gordon road,redland bay,4165
6,1,7,Cannon Hill Fire Station,​Permanent,24 Corporate Dr Cannon Hill 4170,​07 3909 4346,,enquire@cannonhill.qfes.gov.au,-27.46765772,153.0913217,corporate dr cannon hill,corporate dr cannon hill,4170,
7,1,8,Enoggera Fire Station,​Permanent,236 Lloyd St Enoggera 4051,​07 3247 4721,,enquire@enoggera.qfes.gov.au,-27.42545741,152.9888173,lloyd st,lloyd st,enoggera,4051
8,1,9,Camp Hill Fire Station,​Permanent,112 Ferguson Rd Camp Hill 4152,​07 3395 4790,,enquire@camphill.qfes.gov.au,-27.48806998,153.0785562,ferguson rd,ferguson rd,camp hill,4152
9,1,10,Acacia Ridge Fire Station,​Permanent,127 Bradman St Acacia Ridge 4110,​07 3344 4329,,enquire@acaciaridge.qfes.gov.au,-27.58115886,153.0385326,bradman st,bradman st,acacia ridge,4110


In [105]:
print("Hello******")

Hello******


In [107]:
import sys


In [109]:
sys.version

'3.7.3 (default, Mar 27 2019, 16:54:48) \n[Clang 4.0.1 (tags/RELEASE_401/final)]'

In [131]:
final_df.to_csv('result.csv')